# 1. Загрузка данных

## 1.1. Распакуем архив

In [1]:
#вставьте сюда название своего архива вместо 'data_input4.zip'

!unzip data_input4.zip

Archive:  data_input4.zip
   creating: data_input4/
   creating: data_input4/01_healthy/
  inflating: data_input4/01_healthy/cortex_patient13012020_633nm_obj20_power100_1sec_acc40_сenter2900_place1_20201009_1.txt  
  inflating: data_input4/01_healthy/cortex_patient13012020_633nm_obj20_power100_1sec_сenter2900_place1_20201009_1.txt  
  inflating: data_input4/01_healthy/cortex_patient15012020_633nm_obj20_power100_1sec_acc1_сenter2900_place1_20210202.txt  
  inflating: data_input4/01_healthy/cortex_patient15012020_633nm_obj20_power100_1sec_acc1_сenter2900_place2_20210202.txt  
  inflating: data_input4/01_healthy/cortex_patient180219_633nm_obj50_power100_1sec_acc40_сenter2900_place10_20200813_1.txt  
  inflating: data_input4/01_healthy/cortex_patient180219_633nm_obj50_power100_1sec_acc40_сenter2900_place1_20200813_1.txt  
  inflating: data_input4/01_healthy/cortex_patient180219_633nm_obj50_power100_1sec_acc40_сenter2900_place2_20200813_1.txt  
  inflating: data_input4/01_healthy/cortex_pat

## 1.2. Произведем предобработку данных

In [2]:
import os

import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

from collections import Counter

In [3]:
#вставьте название распакованного архова вместо 'data_input4'

root_folder = "data_input4"
all_files = {}
for fld in next(os.walk(root_folder))[1]:
    path = os.path.join(root_folder, fld)
    all_files[fld] = [f for f in os.listdir(path) if os.path.isfile(os.path.join(path, f))]

In [4]:
measurements = []
classifications = []

count_healhy = 0
count_sick = 0

for sample in list(all_files):
    for txt in all_files[sample]:
        if "DS_Store" not in str(txt):
            df = pd.read_csv(os.path.join(root_folder,sample,txt), sep='\t',skiprows=[0],
                header=None, names=[ 'Wave', 'Intensity'])
        #time_unique = df.Wave.unique()
            if len(df[['Intensity']].to_numpy()) != 994:
                measurements.append(df[['Intensity']].to_numpy())
                if 'healthy' in str(sample):
                    classifications.append(0)
                    count_healhy += 1
                if 'sick' in str(sample):
                    classifications.append(1)
                    count_sick += 1

print(f'Здоровые ткани: {count_healhy}')
print(f'Больные ткани: {count_sick}')
print(f'Все ткани: {count_healhy + count_sick}')

Здоровые ткани: 432
Больные ткани: 122
Все ткани: 554


In [5]:
X = np.array(measurements)
y = np.array(classifications)

In [6]:
size = X.shape
X = X.reshape(size[0], size[1])

In [7]:
raman_shifts = pd.read_csv('/content/data_input4/02_sick/image_tumor_patient20022019_633nm_obj20_power100_1sec_сenter2900_step4um_place1_20200929_1_18.txt', sep="\t", header=None)
raman_shifts.drop(columns=[2], inplace = True)
raman_shifts.columns = ["Raman Shifts", "Интенсивность"]
raman_shifts.drop(raman_shifts.head(1).index, inplace=True)
raman_shifts["Raman Shifts"] = pd.to_numeric(raman_shifts["Raman Shifts"], errors='coerce').astype(np.int64)
RS = np.array(raman_shifts["Raman Shifts"])

In [8]:
df = pd.DataFrame(data = X)
df.columns = RS
df = df[RS[::-1]]
target = pd.DataFrame(data = y)
df['target'] = target[0]
df

,2141,2142,2144,2145,2147,2149,2150,2152,2153,2155,...,3540,3541,3542,3544,3545,3546,3547,3549,3550,target
0,63309.828125,64086.429688,63702.246094,63583.066406,63310.402344,63745.718750,63517.710938,63417.527344,63616.433594,63604.218750,...,58740.515625,58647.253906,58971.214844,59289.199219,59328.335938,59009.015625,58995.128906,59024.019531,58455.519531,0
1,29050.279297,28901.376953,29156.480469,29017.107422,29085.402344,29148.933594,29049.414063,29007.412109,28910.996094,28721.722656,...,18337.187500,18400.123047,18033.613281,17982.500000,18189.966797,18112.384766,18161.068359,18393.150391,18042.689453,0
2,20289.597656,20349.181641,20325.742188,20447.669922,20355.525391,20421.574219,20404.517578,20425.832031,20499.945313,20334.078125,...,22137.742188,22059.041016,22397.562500,22007.396484,22407.134766,22000.503906,22433.001953,22256.443359,22203.466797,0
3,31261.214844,31414.677734,31309.474609,31058.833984,31428.085938,31132.582031,30992.009766,30948.916016,30851.408203,31205.072266,...,18664.746094,18424.541016,18560.781250,18450.740234,18819.173828,18513.613281,18631.646484,18389.076172,18702.861328,0
4,39180.527344,39110.910156,39113.128906,39267.121094,39486.710938,38966.312500,38973.277344,38746.742188,38681.660156,39166.988281,...,19086.537109,19066.197266,18935.931641,18638.666016,18946.060547,18834.011719,18911.371094,18958.197266,18760.546875,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
549,54158.031250,53848.007813,53871.640625,53916.050781,54321.640625,54177.558594,54337.191406,54156.242188,54117.574219,54411.710938,...,62946.355469,63041.214844,63404.789063,62732.242188,62855.605469,62606.281250,63479.347656,62777.683594,63205.589844,1
550,269287.562500,269402.531250,268968.125000,269394.562500,269696.562500,269575.093750,269324.031250,268693.812500,270377.937500,270970.062500,...,356921.531250,356540.000000,355901.781250,355633.781250,355671.062500,355918.093750,355022.375000,354615.250000,356648.718750,1
551,24673.197266,24944.699219,24684.478516,24893.812500,24716.535156,24714.990234,24913.298828,24617.533203,24606.355469,24839.986328,...,25523.224609,25093.199219,25407.945313,25122.248047,25207.029297,24984.300781,25079.269531,25145.750000,25382.109375,1
552,25619.833984,25806.953125,25604.455078,25655.869141,25630.601563,25664.458984,25597.605469,25588.289063,25529.375000,25507.230469,...,26337.039063,26635.705078,26582.373047,26166.634766,26349.378906,26224.656250,26372.855469,26496.664063,26600.580078,1


если у вас собственный датафрейм, запустите следующую ячейку

In [9]:
#df = pd.read_csv('your_data.csv')
#df

#2. Предобработаем набор данных

In [10]:
from sklearn.ensemble import RandomForestClassifier

from sklearn.metrics import f1_score
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix 

In [11]:
import pandas as pd
import numpy as np
import pickle

from sklearn.decomposition import PCA

data = df.drop(['target'], axis=1)

pca = 'pca_model.sav'
PCA = pickle.load(open(pca, 'rb'))
sklearn_transformed_data = PCA.transform(data)
_PCA = pd.DataFrame(sklearn_transformed_data)

In [12]:
efs_9 = [2276, 2254, 2282, 2316, 2310, 3273, 2313, 2271, 2322]
rfi_9 = [2186, 2142, 3537, 2177, 2152, 2149, 3544, 2147, 2185]
features_efs_rfi = set(rfi_9 + efs_9)

df_t = df.T
df_ens = df_t.loc[features_efs_rfi]
df_ens = df_ens.T
df_ens['target'] = df.target

<ipython-input-12-a5e3ca8e2125>:6: FutureWarning: Passing a set as an indexer is deprecated and will raise in a future version. Use a list instead.
  df_ens = df_t.loc[features_efs_rfi]


In [13]:
df = pd.concat([_PCA, df_ens], axis=1, join="inner")
df

,0,1,2,3,4,5,6,7,8,9,...,2316,3273,2254,2313,3537,2322,3544,2142,2271,target
0,6.137238e+05,99397.419229,35006.244459,-11774.228038,3585.392532,-2572.756468,2771.012812,28.228479,903.688345,-582.209867,...,64149.503906,62695.359375,63317.265625,64052.343750,58575.062500,64258.480469,59289.199219,64086.429688,63274.289063,0
1,-6.607158e+05,98704.035283,-7985.318865,-4166.961363,-301.106570,353.048293,-28.280044,-479.597871,66.078634,1038.419223,...,27624.705078,20708.824219,28027.146484,27851.978516,18243.931641,27753.585938,17982.500000,28901.376953,27877.017578,0
2,-7.124513e+05,-23063.117397,-7315.160405,2710.570027,-5141.622280,3768.538590,-3466.037890,-1374.407264,-730.074038,-771.040625,...,20533.244141,22408.080078,20364.357422,20706.669922,22293.000000,20472.154297,22007.396484,20349.181641,20368.587891,0
3,-6.340463e+05,116223.419078,-2513.836531,314.154833,-1151.444714,739.879602,-455.738568,-447.504133,78.421535,1277.613541,...,29340.871094,21165.179688,29623.166016,29603.406250,18805.220703,29032.166016,18450.740234,31414.677734,29567.324219,0
4,-5.589717e+05,189487.671332,9837.253004,12936.592289,-1245.303473,2853.039601,-2359.570688,-75.907329,-629.224490,2492.179281,...,35424.363281,21381.466797,36124.078125,35792.210938,18881.101563,35518.511719,18638.666016,39110.910156,36107.980469,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
549,5.658125e+05,-30738.820099,34237.845426,-9561.336441,-758.574322,502.334227,1194.746564,-809.035741,280.712379,-2435.109587,...,56203.285156,65181.488281,55512.972656,55973.851563,62854.351563,56269.503906,62732.242188,53848.007813,55003.937500,1
550,9.422944e+06,-413401.755219,18135.458794,1430.726554,-10876.689875,1904.938558,-14738.956242,3344.645916,255.707601,43745.079600,...,287198.437500,365756.343750,278996.125000,286835.093750,356293.000000,286795.437500,355633.781250,269402.531250,279917.343750,1
551,-5.843075e+05,-4346.092803,-2432.298588,-1879.301738,-3203.479555,2107.297593,-1740.620141,-1036.871892,-327.849803,-612.283499,...,25341.509766,26227.904297,24674.136719,25236.433594,25526.544922,24949.691406,25122.248047,24944.699219,24697.181641,1
552,-5.547086e+05,-5382.646784,54.718127,-1751.490468,-2263.710343,2244.137984,-1613.738611,-939.019266,-257.683309,-876.163815,...,25895.486328,27342.582031,25513.634766,25767.121094,26405.085938,25992.962891,26166.634766,25806.953125,25377.548828,1


In [14]:
from sklearn.metrics import f1_score
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import train_test_split


X = df.drop(['target'], axis=1)
y = df['target']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [15]:
import pickle

_model= 'final_model.sav'
final_model = pickle.load(open(_model, 'rb'))

In [16]:
y_pred = final_model.predict(X_test)

cm_rf = pd.DataFrame(confusion_matrix(y_test, y_pred))
sensitivity = cm_rf[0][0] / (cm_rf[0][0] + cm_rf[1][0])
specificity = cm_rf[1][1] / (cm_rf[0][1] + cm_rf[1][1])

RF_metric = [accuracy_score(y_test, y_pred), f1_score(y_test, y_pred), specificity, sensitivity]
RF_metric

[1.0, 1.0, 1.0, 1.0]

In [17]:
res = pd.DataFrame()
res['Итоговая модель'] = RF_metric
res.index = ['Точность', 'F1 мера', 'Доля верно обнаруженных больных', 'Доля верно обнаруженных здоровых']
res = res.round(3)
res

,Итоговая модель
Точность,1.0
F1 мера,1.0
Доля верно обнаруженных больных,1.0
Доля верно обнаруженных здоровых,1.0


в нашем случае модель дала переобученные результаты, так как не совсем верно предсказывать данные на которых мы обучались, но на ваших данных результаты будут честными ;)